In [1]:
# Imports and setup model
import os
import openai
import json
import configobj
import numpy

config = configobj.ConfigObj('.env')

model = "text-embedding-ada-002" 

openai.api_key = config["OPENAI_API_KEY"]

In [2]:
# generate prompts
text = ""

with open('nodes.md', 'r') as f:
    text = f.read()

print(text)

# `nodes`
The `nodes` group contains data that is typically used to visualize single geographic locations in the "**Map**" view.

The structure of a `nodes` group looks as follows:
```py
"nodes": {
    "types": {
        "customNodeType1": {
            "name": "Node Type A",
            "colorByOptions": {
                "customPropKey1": {
                    "min": 0,
                    "max": 80,
                    "startGradientColor": {
                        "dark": "rgb(233, 0, 0)",
                        "light": "rgb(52, 52, 236)",
                    },
                    "endGradientColor": {
                        "dark": "rgb(96, 2, 2)",
                        "light": "rgb(23, 23, 126)",
                    },
                },
                "customPropKey2": {
                    "min": 0,
                    "max": 50,
                    "startGradientColor": {
                        "dark": "rgb(233, 0, 0)",
                        "light": "rgb(52, 52, 2

In [4]:
# get completions
results = []
result = openai.Embedding.create(
model=model,
input=text,
)
  # add stop="end" for fine-tuned
results.append(result)

print(results)

[[<OpenAIObject list at 0x7f83d8128410> JSON: {
  "object": "list",
  "data": [
    {
      "object": "embedding",
      "index": 0,
      "embedding": [
        -0.0007825489738024771,
        0.01413770578801632,
        -0.006039274390786886,
        -0.030901184305548668,
        -0.04842475429177284,
        0.03222788870334625,
        -0.010903861373662949,
        -0.007711476180702448,
        0.01381985004991293,
        -0.03501949831843376,
        0.007884223945438862,
        0.011014420539140701,
        -0.042703334242105484,
        0.015934286639094353,
        0.010371796786785126,
        0.013453623279929161,
        0.009895012713968754,
        -0.03537881374359131,
        0.037451792508363724,
        0.007621646858751774,
        0.009521876461803913,
        0.017026055604219437,
        -0.03120522014796734,
        -0.02030135877430439,
        0.005382831208407879,
        0.02067449502646923,
        0.01560261007398367,
        -0.03689899668097496,
    

In [6]:
print(results[0][0]['data'][0]['embedding'])

[-0.0007825489738024771, 0.01413770578801632, -0.006039274390786886, -0.030901184305548668, -0.04842475429177284, 0.03222788870334625, -0.010903861373662949, -0.007711476180702448, 0.01381985004991293, -0.03501949831843376, 0.007884223945438862, 0.011014420539140701, -0.042703334242105484, 0.015934286639094353, 0.010371796786785126, 0.013453623279929161, 0.009895012713968754, -0.03537881374359131, 0.037451792508363724, 0.007621646858751774, 0.009521876461803913, 0.017026055604219437, -0.03120522014796734, -0.02030135877430439, 0.005382831208407879, 0.02067449502646923, 0.01560261007398367, -0.03689899668097496, -0.0014173983363434672, -0.02949155867099762, 0.025428522378206253, -0.008685775101184845, -0.03427322581410408, -0.025815479457378387, -0.002005605725571513, -0.015934286639094353, 0.004850767087191343, -0.03504713997244835, 0.022097939625382423, 0.014593761414289474, -0.009362948127090931, 0.008457748219370842, 0.0016678831307217479, -0.04179122671484947, 0.0011202715104445815

In [8]:
# save results
with open('embedding_results_standard.jsonl', 'w') as f:
  for item in results:
    f.write(json.dumps(item[0]['data'][0]['embedding']) + '\n')